# Activity 2.2 - Transfer Learning 

#### Objective(s):

This activity aims to introduce how to apply transfer learning 

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network 
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [2]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 20
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [3]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps. 
* Use the model, training set, test set and number of classes as function parameters


In [4]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [5]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()



Create two datasets 
* one with digits below 5
* one with 5 and above

In [6]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [7]:


feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [8]:


classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [9]:

model = Sequential(feature_layers + classification_layers)

Check the model summary

In [10]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [11]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/20


230/230 [==============================] - 6s 13ms/step - loss: 1.6051 - accuracy: 0.2114 - val_loss: 1.5870 - val_accuracy: 0.2897
Epoch 2/20
230/230 [==============================] - 2s 11ms/step - loss: 1.5835 - accuracy: 0.2746 - val_loss: 1.5632 - val_accuracy: 0.4583
Epoch 3/20
230/230 [==============================] - 2s 11ms/step - loss: 1.5612 - accuracy: 0.3359 - val_loss: 1.5380 - val_accuracy: 0.5361
Epoch 4/20
230/230 [==============================] - 2s 11ms/step - loss: 1.5382 - accuracy: 0.3902 - val_loss: 1.5101 - val_accuracy: 0.5647
Epoch 5/20
230/230 [==============================] - 2s 11ms/step - loss: 1.5125 - accuracy: 0.4436 - val_loss: 1.4782 - val_accuracy: 0.5980
Epoch 6/20
230/230 [==============================] - 2s 10ms/step - loss: 1.4821 - accuracy: 0.4816 - val_loss: 1.4414 - val_accuracy: 0.6386
Epoch 7/20
230/230 [==============================] - 2s 10ms/step - loss: 1.4463 - accuracy: 0.5293 - val_loss: 1.3987 - val_accuracy: 0.6750
Epoch 8/20

Freeze only the feature layers

In [12]:

for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [14]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/20


240/240 [==============================] - 2s 7ms/step - loss: 1.6710 - accuracy: 0.3730 - val_loss: 1.5124 - val_accuracy: 0.4227
Epoch 2/20
240/240 [==============================] - 1s 6ms/step - loss: 1.5323 - accuracy: 0.4125 - val_loss: 1.3775 - val_accuracy: 0.4631
Epoch 3/20
240/240 [==============================] - 1s 6ms/step - loss: 1.4020 - accuracy: 0.4521 - val_loss: 1.2619 - val_accuracy: 0.5052
Epoch 4/20
240/240 [==============================] - 1s 6ms/step - loss: 1.3032 - accuracy: 0.4911 - val_loss: 1.1606 - val_accuracy: 0.5635
Epoch 5/20
240/240 [==============================] - 2s 7ms/step - loss: 1.2125 - accuracy: 0.5327 - val_loss: 1.0679 - val_accuracy: 0.6313
Epoch 6/20
240/240 [==============================] - 1s 6ms/step - loss: 1.1264 - accuracy: 0.5818 - val_loss: 0.9853 - val_accuracy: 0.6871
Epoch 7/20
240/240 [==============================] - 1s 6ms/step - loss: 1.0480 - accuracy: 0.6281 - val_loss: 0.9105 - val_accuracy: 0.7334
Epoch 8/20
240/24

#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [15]:
feature_layers_2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers_2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

model2 = Sequential(feature_layers_2 + classification_layers_2)

In [16]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [17]:
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/20
240/240 [==============================] - 4s 13ms/step - loss: 1.6055 - accuracy: 0.2245 - val_loss: 1.5869 - val_accuracy: 0.3744
Epoch 2/20
240/240 [==============================] - 3s 12ms/step - loss: 1.5807 - accuracy: 0.2994 - val_loss: 1.5594 - val_accuracy: 0.5431
Epoch 3/20
240/240 [==============================] - 3s 12ms/step - loss: 1.5549 - accuracy: 0.3801 - val_loss: 1.5287 - val_accuracy: 0.6602
Epoch 4/20
240/240 [==============================] - 3s 11ms/step - loss: 1.5238 - accuracy: 0.4651 - val_loss: 1.4927 - val_accuracy: 0.7357
Epoch 5/20
240/240 [==============================] - 3s 13ms/step - loss: 1.4882 - accuracy: 0.5337 - val_loss: 1.4495 - val_accuracy: 0.7953
Epoch 6/20
240/240 [==============================] - 3s 12ms/step - loss: 1.4448 - accuracy: 0.5986 - val_loss: 1.3970 - val_accuracy: 0.8416
Epoch 7/20
240/240 [==============================] - 3s 11ms/step - l

In [18]:
for l in feature_layers_2:
    l.trainable = False

In [19]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [20]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/20


230/230 [==============================] - 2s 6ms/step - loss: 1.9151 - accuracy: 0.3231 - val_loss: 1.7041 - val_accuracy: 0.3742
Epoch 2/20
230/230 [==============================] - 1s 6ms/step - loss: 1.7384 - accuracy: 0.3474 - val_loss: 1.5365 - val_accuracy: 0.4014
Epoch 3/20
230/230 [==============================] - 2s 7ms/step - loss: 1.5852 - accuracy: 0.3809 - val_loss: 1.3923 - val_accuracy: 0.4409
Epoch 4/20
230/230 [==============================] - 2s 7ms/step - loss: 1.4525 - accuracy: 0.4207 - val_loss: 1.2658 - val_accuracy: 0.5007
Epoch 5/20
230/230 [==============================] - 2s 8ms/step - loss: 1.3386 - accuracy: 0.4641 - val_loss: 1.1566 - val_accuracy: 0.5703
Epoch 6/20
230/230 [==============================] - 2s 7ms/step - loss: 1.2364 - accuracy: 0.5149 - val_loss: 1.0623 - val_accuracy: 0.6314
Epoch 7/20
230/230 [==============================] - 2s 7ms/step - loss: 1.1572 - accuracy: 0.5601 - val_loss: 0.9826 - val_accuracy: 0.6776
Epoch 8/20
230/23

#### Conclusion

#type your answer here

In this activity, I learned about the concept of transfer learning in neural networks. Transfer learning is a technique used in neural networks where it reuses the knowledge gained from a previously trained model on a new model. This technique is useful because it can reduce training time and improved the performance of the next model to be trained.